[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/CCS-ZCU/pribehy-dat/blob/master/scripts/abm.ipynb)

# ABM: Modelování založené na agentech

**autor**: *Vojtěch Kaše* (kase@ff.zcu.cz)

[![](https://ccs.zcu.cz/wp-content/uploads/2021/10/cropped-ccs-logo_black_space_240x240.png)](https://ccs.zcu.cz)

## Úvod a cíle kapitoly

Toto cvičení nás uvádí do metodologie sociálních simulací, nebo úžeji ABM (=agent based modeling). Konkrétně budeme pracovat s jedním velice známým modelem, který navhrl již počátkem 70. let americký ekonom [Thomas Schelling](https://en.wikipedia.org/wiki/Thomas_Schelling). Tento model na velice abstraktní úrovni ukazuje, jak i v zásadě tolerantní prostředí může vést k segregaci společnosti. Základem modelu je množina agentů (lidí), kteří jsou rozmístěni v určitém prostoru ve formě jakési mřížky či šachovnice. Základní vlastností každého agenta je informace o tom, kolik jeho sousedů (agentů na okolních buňkách mřížky) je ze stejné nebo odlišné skupiny než on, což je typicky odlišeno barvou. Tato informace je základem binárního atributu spokojenosti (happiness): Agent je spokojený tehdy, pokud je alespoň určité procento jeho sousedů (např. 30 %) ze stejné skupiny (barvy) jako on.

Model poté probíhá v několika po sobě jdoucích krocích:
* Inicializace: Agenti jsou náhodně rozmístěni v prostoru.
* Akce: Každý agent zvažuje své okolí a rozhoduje se, zda je spokojen se svým umístěním. Pokud je spokojen, nic nedělá. Pokud není spokojen, náhodně se přesouvá na jiné volné místo.
* Aktualizace: Provede se aktualizace stavu prostředí.

Tento proces se opakuje, dokud není dosaženo stabilního stavu, kdy už žádný agent nemá potřebu se přesunout, tj. všichni agenti jsou spokojeni. Výsledkem tohoto modelu může být zajímavé chování, kde malé individuální preference vedou k výrazné segregaci v populaci, přestože žádný jednotlivec nemá silnou preferenci pro segregaci.

Tento model je ve známé podobě implementován v programu netlogo ([Segregation](https://www.netlogoweb.org/launch#https://www.netlogoweb.org/assets/modelslib/Sample%20Models/Social%20Science/Segregation.nlogo)), kde je možné si k němu načíst dovysvětlující informace.

Na rozdíl od jiných cvičení v této sestavě elektronických opor, toto cvičení je přejato téměř beze změny z dokumentace ke knihovně [agentpy](https://agentpy.readthedocs.io) (viz [zde](https://agentpy.readthedocs.io/en/latest/agentpy_segregation.html)), odkud je převzat i průvodní text v angličtině.

In [ ]:
%%capture
!pip install agentpy
import os
#os._exit(00)

In [ ]:
# Model design
import agentpy as ap
import random

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
from IPython.display import display

## About the model

The model is based on the [NetLogo Segregation model](http://ccl.northwestern.edu/netlogo/models/Segregation) from Uri Wilensky, who describes it as follows:

>This project models the behavior of two types of agents in a neighborhood. The orange agents and blue agents get along with one another. But each agent wants to make sure that it lives near some of "its own." That is, each orange agent wants to live near at least some orange agents, and each blue agent wants to live near at least some blue agents. The simulation shows how these individual preferences ripple through the neighborhood, leading to large-scale patterns.

## Model definition

To start, we define our agents, who initiate with a random group
and have two methods to check whether they are happy and to
move to a new location if they are not.

In [ ]:
class Person(ap.Agent):

    def setup(self):
        """ Initiate agent attributes. """
        self.grid = self.model.grid
        self.random = self.model.random
        self.group = self.random.choice(range(self.p.n_groups))
        self.share_similar = 0
        self.happy = False

    def update_happiness(self):
        """ Be happy if rate of similar neighbors is high enough. """
        neighbors = self.grid.neighbors(self)
        similar = len([n for n in neighbors if n.group == self.group])
        ln = len(neighbors)
        self.share_similar = similar / ln if ln > 0 else 0
        self.happy = self.share_similar >= self.p.want_similar

    def find_new_home(self):
        """ Move to random free spot and update free spots. """
        new_spot = self.random.choice(self.model.grid.empty)
        self.grid.move_to(self, new_spot)

Next, we define our model, which consists of our agens and a spatial grid environment.
At every step, unhappy people move to a new location.
After every step (update), agents update their happiness.
If all agents are happy, the simulation is stopped.

In [ ]:
class SegregationModel(ap.Model):

    def setup(self):

        # Parameters
        s = self.p.size
        n = self.n = int(self.p.density * (s ** 2))

        # Create grid and agents
        self.grid = ap.Grid(self, (s, s), track_empty=True)
        self.agents = ap.AgentList(self, n, Person)
        self.grid.add_agents(self.agents, random=True, empty=True)

    def update(self):
        # Update list of unhappy people
        self.agents.update_happiness()
        self.unhappy = self.agents.select(self.agents.happy == False)

        # Stop simulation if all are happy
        if len(self.unhappy) == 0:
            self.stop()

    def step(self):
        # Move unhappy people to new location
        self.unhappy.find_new_home()

    def get_segregation(self):
        # Calculate average percentage of similar neighbors
        return round(sum(self.agents.share_similar) / self.n, 2)

    def end(self):
        # Measure segregation at the end of the simulation
        self.report('segregation', self.get_segregation())

In [ ]:
# Tato buňka slouží ke kontrole průchodu tímto cvičením. 
# Pokud toto cvičení plníte v rámci svých studijních povinností na ZČU, buňku spusťte a držte se instrukcí.
exec(requests.get("https://sciencedata.dk/shared/856b0a7402aa7c7258186a8bdb329bd3?download").text)
kontrola_pruchodu(ntb="abm", arg1=1)

## Single-run animation

Uri Wilensky explains the dynamic of the segregation model as follows:

>Agents are randomly distributed throughout the neighborhood. But many agents are "unhappy" since they don't have enough same-color neighbors. The unhappy agents move to new locations in the vicinity. But in the new locations, they might tip the balance of the local population, prompting other agents to leave. If a few agents move into an area, the local blue agents might leave. But when the blue agents move to a new area, they might prompt orange agents to leave that area.
>
>Over time, the number of unhappy agents decreases. But the neighborhood becomes more segregated, with clusters of orange agents and clusters of blue agents.
>
>In the case where each agent wants at least 30% same-color neighbors, the agents end up with (on average) 70% same-color neighbors. So relatively small individual preferences can lead to significant overall segregation.

To observe this effect in our model, we can create an animation of a single run.  
To do so, we first set up an instance of our model with a chosen set of parameters.

In [ ]:
parameters = {
    'want_similar': 0.3, # For agents to be happy
    'n_groups': 2, # Number of groups
    'density': 0.95, # Density of population
    'size': 100, # Height and length of the grid
    'steps': 100  # Maximum number of steps
    }

We can now create an animation plot and display it directly in Jupyter as follows.

In [ ]:
def animation_plot(model, ax):
    group_grid = model.grid.attr_grid('group')
    ap.gridplot(group_grid, cmap='Accent', ax=ax)
    ax.set_title(f"Segregation model \n Time-step: {model.t}, "
                 f"Segregation: {model.get_segregation()}")

fig, ax = plt.subplots()
model = SegregationModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
display(IPython.display.HTML(animation.to_jshtml()))

## Multi-run experiment

To explore how different individual preferences lead to different average levels of segregation, we can conduct a multi-run experiment.
To do so, we first prepare a parameter sample that includes different values for peoples' preferences and the population density.

In [ ]:
parameters_multi = dict(parameters)
parameters_multi.update({
    'want_similar': ap.Values(0,0.125, 0.25, 0.375, 0.5, 0.625),
    'density': ap.Values(0.5, 0.7, 0.95),
})
sample = ap.Sample(parameters_multi)

We now run an experiment where we simulate each parameter combination in our sample over 5 iterations.

In [ ]:
exp = ap.Experiment(SegregationModel, sample, iterations=5)
results = exp.run()

Finally, we can arrange the results from our experiment into a dataframe with measures and variable parameters,
and use the seaborn library to visualize the different segregation levels over our parameter ranges.

In [ ]:
sns.set_theme()
sns.lineplot(
    data=results.arrange_reporters(),
    x='want_similar',
    y='segregation',
    hue='density'
);

In [ ]:
exec(requests.get("https://sciencedata.dk/shared/856b0a7402aa7c7258186a8bdb329bd3?download").text)
kontrola_pruchodu(ntb="abm", arg1=2)